Давай напишем спарк код для этого запроса?!

event_id - порядковый

Как мы его напишем?
1. Написать несколько тестовых случаев
2. Пронумеровать все ноды и условия в запросе
3. Для каждого пользователя построить фрейм цепочек его событий
4. Таблица совпадений нод и ивентов - запись есть в том случае, если нода подходит под условия
    user_id, event_id, node_id

5. Таблица условий
    Если - event_id = node_id, то какие ограничения накладываются на другие вершины
    user_id, event_id, node_id, condition_node_id, [event_ids]

    Дефолтные ограничения - event_id должен просто быть больше


6. Последовательность джойнов для каждой следующей пары нод. Цепочки на выходе - ответ

In [4]:
import pandas as pd

In [5]:
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

athena_conn = connect(
    s3_staging_dir="s3://idle-tycoon-adjust/athena-query-results/", 
    region_name="eu-central-1",
    cursor_class=PandasCursor
)

In [14]:
events_df = pd.read_sql("""
SELECT *
FROM analytics.raw_events
WHERE date(event_timestamp) BETWEEN date('2023-10-17') AND date('2023-10-19')
    AND game_name = 'Match10 Jam'
ORDER BY user_id, event_timestamp
""", athena_conn)

In [15]:
events_df.to_csv('events-test-sample.csv', index=False)

In [6]:
events_df = pd.read_csv('events-test-sample.csv')

# Query example

In [7]:
query = [
    {
        'node_conditions': ['type == "gameStarted"'],
        'order_conditions': []
    },
    {
        'node_conditions': ['type == "LevelFunnelStart"'],
        'order_conditions': [{
            'node_used': 0,
            'condition': 'abs(time_A - time_B) <= 20'
        }]
    }, 
    {
        'not': [
            {
                'node_conditions': ['type == "LevelFunnelEnd"'],
            }
        ]
    },
    {
        'node_conditions': ['type == "userDropped"']
    }
]

# Events preprocessing

## Step 1: node conditions

In [8]:
query_events_df = events_df[['event_timestamp', 'user_id', 'event_name', 'session_id']].rename(columns={
    'event_name': 'type'
}).copy()

In [9]:
query_events_df['event_order_id'] = query_events_df.groupby(['user_id']).transform('cumcount')

In [10]:
query_events_df['event_index'] = query_events_df['user_id'].astype(str) + '_' + query_events_df['event_order_id'].astype(str)

In [11]:
query_events_df = query_events_df.set_index('event_index')

In [14]:
query_events_df['event_timestamp'] = pd.to_datetime(query_events_df['event_timestamp'])

In [15]:
query_events_df['time'] = query_events_df['event_timestamp'].astype(int) // 1000 // 1000 // 1000

/tmp/ipykernel_5227/566312693.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  query_events_df['time'] = query_events_df['event_timestamp'].astype(int) // 1000 // 1000 // 1000


In [16]:
def check_node_condition(event_row, condition):
    return eval(condition, {
        'this': event_row
    })

In [17]:
answers = []

for query_node_id, node in enumerate(query):
    for node_condition_id, condition in enumerate(node['node_conditions']):
        answer_df = query_events_df[['user_id', 'event_order_id']].copy()
        answer_df['answer'] = query_events_df.apply(lambda x: check_node_condition(x, condition), axis=1).to_frame()
        answer_df['query_node_id'] = query_node_id
        answer_df['node_condition_id'] = node_condition_id
        
        answers.append(answer_df[answer_df['answer'] == True])

In [18]:
node_answers_df = pd.concat(answers)

In [19]:
node_answers_df.head()

,user_id,event_order_id,answer,query_node_id,node_condition_id
event_index,,,,,
0034d8683132554bdb53261a8998ec48_2,0034d8683132554bdb53261a8998ec48,2,True,0,0
01b410469c2f68c1a8d605d9cb4c6132_1,01b410469c2f68c1a8d605d9cb4c6132,1,True,0,0
021fe6b20d2504086a079cf4f58d3a9c_1,021fe6b20d2504086a079cf4f58d3a9c,1,True,0,0
025911546f86c04738fd77cd44f46e55_2,025911546f86c04738fd77cd44f46e55,2,True,0,0
025911546f86c04738fd77cd44f46e55_56,025911546f86c04738fd77cd44f46e55,56,True,0,0


## Step 2: order conditions

In [20]:
node_answers_df[node_answers_df['query_node_id'] == query_node_id]

,user_id,event_order_id,answer,query_node_id,node_condition_id
event_index,,,,,
01b410469c2f68c1a8d605d9cb4c6132_4,01b410469c2f68c1a8d605d9cb4c6132,4,True,1,0
01b410469c2f68c1a8d605d9cb4c6132_6,01b410469c2f68c1a8d605d9cb4c6132,6,True,1,0
01b410469c2f68c1a8d605d9cb4c6132_8,01b410469c2f68c1a8d605d9cb4c6132,8,True,1,0
01b410469c2f68c1a8d605d9cb4c6132_10,01b410469c2f68c1a8d605d9cb4c6132,10,True,1,0
01b410469c2f68c1a8d605d9cb4c6132_12,01b410469c2f68c1a8d605d9cb4c6132,12,True,1,0
...,...,...,...,...,...
ff7e82fd53b445ff681fe0452b14812c_31,ff7e82fd53b445ff681fe0452b14812c,31,True,1,0
ff7e82fd53b445ff681fe0452b14812c_36,ff7e82fd53b445ff681fe0452b14812c,36,True,1,0
ff7e82fd53b445ff681fe0452b14812c_38,ff7e82fd53b445ff681fe0452b14812c,38,True,1,0


Для каждого условия мне нужно указать, какой именно ноды запроса оно касается

In [66]:
def check_node_condition(event_row, condition):
    return eval(condition, {
        'this': event_row
    })

In [178]:
def get_events_by_condition(conditions):
    global query_events_df
    
    condition = " & ".join([f"({c})" for c in conditions])
    
    selected_events = query_events_df.query(condition).index
    
    return query_events_df.loc[selected_events, ['user_id', 'event_order_id']]

In [68]:
def merge_new_nodes(sequences_df, new_nodes_df, index):
    return sequences_df.merge(new_nodes_df, how='inner', on=['user_id']).query(
        f'(event_{index - 1} < event_{index})'
    )

In [109]:
def get_events_index(user_ids, event_ids):
    return user_ids.astype(str).values + '_' + event_ids.astype(str).values

In [151]:
def get_pairs_df(sequences_df, current_node, target_node):
    global query_events_df
    events_A_index = get_events_index(sequences_df['user_id'], sequences_df[f'event_{target_node}'])
    events_B_index = get_events_index(sequences_df['user_id'], sequences_df[f'event_{current_node}'])
    events_A_df = query_events_df.loc[events_A_index].reset_index(drop=True)
    events_B_df = query_events_df.loc[events_B_index].reset_index(drop=True)

    events_A_df.columns = [f'{c}_A' for c in events_A_df.columns]
    events_B_df.columns = [f'{c}_B' for c in events_B_df.columns]
    event_pairs_df = pd.concat([events_A_df, events_B_df], axis=1)
    event_pairs_df.index = sequences_df.index
    
    return event_pairs_df

In [153]:
def select_pairs_df(event_pairs_df, condition):
    return event_pairs_df.query(condition)

In [170]:
# query = [
#     {
#         'node_conditions': ['type == "gameStarted"'],
#         'order_conditions': []
#     },
#     {
#         'node_conditions': ['type == "LevelFunnelStart"'],
#         'order_conditions': [{
#             'node_used': 0,
#             'condition': 'abs(time_A - time_B) >= 500'
#         }]
#     }
# ]

In [225]:
query = [
    {
        'node_conditions': ['type == "LevelFunnelStart"']
    },
    {
        'node_conditions': ['type == "gameStarted"'],
        'order_conditions': [{
            'node_used': 0,
            'condition': '(abs(time_A - time_B) <= 60 * 2) & (session_id_A != session_id_B)'
        }]
    },
]

In [195]:
query = [
    {
        'node_conditions': ['type == "LevelFunnelStart"']
    },
    {
        'node_conditions': ['type == "gameStarted"'],
        'order_conditions': [{
            'node_used': 0,
            'condition': '(abs(time_A - time_B) <= 60 * 2) & (session_id_A != session_id_B)'
        }]
    },
    {
        'node_conditions': ['type == "LevelFunnelEnd"'],
        'order_conditions': []
    }
]

In [196]:
sequences_df = get_events_by_condition(query[0]['node_conditions']).reset_index(drop=True).rename(columns={
    'event_order_id': 'event_0'
})

In [197]:
# sequences_df

In [198]:
for index, query_node in enumerate(query[1:]):
    new_nodes_df = get_events_by_condition(query_node['node_conditions']).rename(columns={
        'event_order_id': f'event_{index + 1}'
    })
    # Cross join + remove new id <= old id
    sequences_df = merge_new_nodes(sequences_df, new_nodes_df, index + 1) 
        
    for order_condition in query_node['order_conditions']:
        target_node = order_condition['node_used']
        event_pairs_df = get_pairs_df(sequences_df, index + 1, target_node)
        event_pairs_df = select_pairs_df(event_pairs_df, order_condition['condition'])
        sequences_df = sequences_df.loc[event_pairs_df.index].copy()

In [199]:
# sequences_total_df = sequences_df.copy()
sequences_subtract_df = sequences_df.copy()

In [201]:
sequences_total_df

,user_id,event_0,event_1
434,140116b267f729f9394d2ef71e1c7173,24,29
437,140116b267f729f9394d2ef71e1c7173,26,29
440,140116b267f729f9394d2ef71e1c7173,28,29
516,15ac049c129c60095a87dc41be1c04a2,40,44
519,15ac049c129c60095a87dc41be1c04a2,42,44
2027,445a04a15eee25fbe85b2252bb20f1ea,4,9
2029,445a04a15eee25fbe85b2252bb20f1ea,6,9
2047,478d57316d154ef88816eb9617bda598,20,27
2049,478d57316d154ef88816eb9617bda598,22,27
2051,478d57316d154ef88816eb9617bda598,24,27


In [ ]:
sequences_total_df

In [215]:
sequences_subtract_i_df = sequences_subtract_df[['event_0', 'event_1']].drop_duplicates()

In [217]:
sequences_subtract_i_df['remove'] = True

In [219]:
total_sequences_with_marks_df = sequences_total_df.merge(sequences_subtract_i_df, on=['event_0', 'event_1'], how='left')

In [223]:
sequences_df = total_sequences_with_marks_df[
    total_sequences_with_marks_df['remove'] != True
].copy()

In [247]:
sequences_df

,user_id,event_0,event_1,remove
0,140116b267f729f9394d2ef71e1c7173,24,29,NaN
1,140116b267f729f9394d2ef71e1c7173,26,29,NaN
2,140116b267f729f9394d2ef71e1c7173,28,29,NaN
3,15ac049c129c60095a87dc41be1c04a2,40,44,NaN
4,15ac049c129c60095a87dc41be1c04a2,42,44,NaN
5,445a04a15eee25fbe85b2252bb20f1ea,4,9,NaN
6,445a04a15eee25fbe85b2252bb20f1ea,6,9,NaN
7,478d57316d154ef88816eb9617bda598,20,27,NaN
8,478d57316d154ef88816eb9617bda598,22,27,NaN
9,478d57316d154ef88816eb9617bda598,24,27,NaN


,event_timestamp,user_id,type,session_id,event_order_id,time
event_index,,,,,,
478d57316d154ef88816eb9617bda598_24,2023-10-17 12:06:35.056,478d57316d154ef88816eb9617bda598,LevelFunnelStart,e7fa01ee-f9f3-487c-ac72-26775fab8b50,24,1697544395
478d57316d154ef88816eb9617bda598_25,2023-10-17 12:07:21.101,478d57316d154ef88816eb9617bda598,sdkStart,6e1ab510-4cfc-4c28-9827-e8f682be1de7,25,1697544441
478d57316d154ef88816eb9617bda598_26,2023-10-17 12:07:21.101,478d57316d154ef88816eb9617bda598,clientDevice,6e1ab510-4cfc-4c28-9827-e8f682be1de7,26,1697544441
478d57316d154ef88816eb9617bda598_27,2023-10-17 12:07:21.102,478d57316d154ef88816eb9617bda598,gameStarted,6e1ab510-4cfc-4c28-9827-e8f682be1de7,27,1697544441


In [246]:
row = sequences_df.iloc[6]
user_id = row['user_id']
start_id = row['event_0'] - 1
end_id = row[f'event_{len(query) - 1}'] + 1
query_events_df.query(f'(user_id == "{user_id}") & (event_order_id >= {start_id}) & (event_order_id <= {end_id})')

,event_timestamp,user_id,type,session_id,event_order_id,time
event_index,,,,,,
445a04a15eee25fbe85b2252bb20f1ea_5,2023-10-17 14:43:50.662,445a04a15eee25fbe85b2252bb20f1ea,LevelFunnelEnd,7fb2de17-4105-463a-9da7-b2da593b0643,5,1697553830
445a04a15eee25fbe85b2252bb20f1ea_6,2023-10-17 14:43:50.668,445a04a15eee25fbe85b2252bb20f1ea,LevelFunnelStart,7fb2de17-4105-463a-9da7-b2da593b0643,6,1697553830
445a04a15eee25fbe85b2252bb20f1ea_7,2023-10-17 14:45:00.722,445a04a15eee25fbe85b2252bb20f1ea,sdkStart,a925433c-5416-4b5c-89b4-186f3d5fc21d,7,1697553900
445a04a15eee25fbe85b2252bb20f1ea_8,2023-10-17 14:45:00.722,445a04a15eee25fbe85b2252bb20f1ea,clientDevice,a925433c-5416-4b5c-89b4-186f3d5fc21d,8,1697553900
445a04a15eee25fbe85b2252bb20f1ea_9,2023-10-17 14:45:00.723,445a04a15eee25fbe85b2252bb20f1ea,gameStarted,a925433c-5416-4b5c-89b4-186f3d5fc21d,9,1697553900


Движок есть +-, теперь нужен генератор запросов. Какие функции нужны?

- Отрицание (для вершины или для сегмента):
    - Выполняется запрос без сегмента
    - Выполняется запрос с сегментом
    - Вычитается одно из другого
    
Если в запросе несколько отрицаний - выполняем без всех, а затем с каждым отдельно!

Сделаем первый джойн, проверим условия

In [100]:
node_A_df = node_answers_df[
    node_answers_df['query_node_id'] == 0
].copy()

In [101]:
node_B_df = node_answers_df[
    node_answers_df['query_node_id'] == 1
].copy()

In [105]:
joined_frame_df = node_A_df.merge(
    node_B_df,
    on=['user_id'],
    suffixes=('_A', '_B')
)

In [108]:
joined_frame_df = joined_frame_df[
    joined_frame_df['event_order_id_B'] >= joined_frame_df['event_order_id_A']
].copy()

In [109]:
# joined_frame_df[
#     joined_frame_df['event_order_id_A'] >= event_order_id_A['event_order_id_B']
# ]

,user_id,event_order_id_A,answer_A,query_node_id_A,node_condition_id_A,event_order_id_B,answer_B,query_node_id_B,node_condition_id_B
0,4033d8b2c2c002310894fdfc6dcc0341,2,True,0,0,4,True,1,0
2,5b37e4620105024f3b30f1cee7dbab2a,2,True,0,0,4,True,1,0
3,5fce4ad4020adbc529afde5cd64d33e9,2,True,0,0,4,True,1,0
5,99934bb795cd816808755ff0257a15f5,1,True,0,0,4,True,1,0
10,ddab633b3b486cf3bb5f1607a6efa512,2,True,0,0,4,True,1,0


In [133]:
events_A_index = joined_frame_df['user_id'].astype(str) + '_' + joined_frame_df['event_order_id_A'].astype(str)

In [134]:
events_B_index = joined_frame_df['user_id'].astype(str) + '_' + joined_frame_df['event_order_id_B'].astype(str)

In [143]:
events_A =  query_events_df.loc[events_A_index].reset_index(drop=True)

In [144]:
events_B = query_events_df.loc[events_B_index].reset_index(drop=True)

In [147]:
events_A.columns = [c + '_A' for c in events_A.columns]
events_B.columns = [c + '_B' for c in events_B.columns]

In [148]:
event_pairs_df = pd.concat([
    events_A,
    events_B
], axis=1)

In [156]:
order_condition = query[1]['order_conditions'][0]['condition']

In [159]:
event_pairs_df['answer'] = event_pairs_df.apply(lambda x: check_node_condition(x, order_condition), axis=1)

In [161]:
event_pairs_df

,event_timestamp_A,user_id_A,type_A,session_id_A,event_order_id_A,time_A,event_timestamp_B,user_id_B,type_B,session_id_B,event_order_id_B,time_B,answer
0,2023-10-19 10:05:40.854,4033d8b2c2c002310894fdfc6dcc0341,gameStarted,b22432f4-cc1d-49fb-a649-a568fe6a4b1a,2,1697709940,2023-10-19 10:05:40.936,4033d8b2c2c002310894fdfc6dcc0341,gameEnded,4ac4a9ed-379a-4b40-a274-4f9fe2b01309,4,1697709940,True
1,2023-10-19 13:48:29.047,5b37e4620105024f3b30f1cee7dbab2a,gameStarted,4e4893ad-26c2-4521-960c-7af3cdb34797,2,1697723309,2023-10-19 13:48:30.833,5b37e4620105024f3b30f1cee7dbab2a,gameEnded,8742f7e3-fcf3-4f1d-a28e-c49cbc93ab26,4,1697723310,True
2,2023-10-18 15:08:52.599,5fce4ad4020adbc529afde5cd64d33e9,gameStarted,bf16a58b-d6dc-42a5-8e6a-64a49d24e604,2,1697641732,2023-10-18 15:08:52.732,5fce4ad4020adbc529afde5cd64d33e9,gameEnded,9fddb894-0640-447b-953e-c723da2c2ea6,4,1697641732,True
3,2023-10-17 12:29:13.029,99934bb795cd816808755ff0257a15f5,gameStarted,e7a2a9ba-e57c-40af-814d-f69f45865270,1,1697545753,2023-10-17 12:29:13.372,99934bb795cd816808755ff0257a15f5,gameEnded,db0dfa2e-d3b8-4159-a568-c3ed2fe2aa35,4,1697545753,True
4,2023-10-18 12:35:34.405,ddab633b3b486cf3bb5f1607a6efa512,gameStarted,5b3dcda7-7c96-49c3-85c2-9e49c45f5fcf,2,1697632534,2023-10-18 12:35:34.642,ddab633b3b486cf3bb5f1607a6efa512,gameEnded,5b3dcda7-7c96-49c3-85c2-9e49c45f5fcf,4,1697632534,True


In [117]:
events_B_df = joined_frame_df[['user_id', 'event_order_id_B']].copy()

In [120]:
# query_events_df

In [121]:
indexed_query_events_df = query_events_df.set_index(['user_id', 'event_order_id'])

In [123]:
indexed_query_events_df.loc[events_B_df.values]

ValueError: Cannot index with multidimensional key

,user_id,event_order_id_B
0,4033d8b2c2c002310894fdfc6dcc0341,4
2,5b37e4620105024f3b30f1cee7dbab2a,4
3,5fce4ad4020adbc529afde5cd64d33e9,4
5,99934bb795cd816808755ff0257a15f5,4
10,ddab633b3b486cf3bb5f1607a6efa512,4


In [91]:
answers = []

for query_node_id, node in enumerate(query):
    suitable_nodes_df = node_answers_df[node_answers_df['query_node_id'] == query_node_id]
    for order_condition_id, condition in enumerate(node['order_conditions']):
        
#         answer_df = query_events_df[['user_id', 'event_order_id']].copy()
#         answer_df['answer'] = query_events_df.apply(lambda x: check_node_condition(x, condition), axis=1).to_frame()
#         answer_df['query_node_id'] = query_node_id
#         answer_df['node_condition_id'] = node_condition_id
        
#         answers.append(answer_df[answer_df['answer'] == True])

0


Начало:
- Выбираем вершину 0
- Начинаем от нее последовательность

Далее:
- Выбираем вершину i + 1
- Мержим с таблицей последовательностей, выбирая только правильные строки
- Для каждого порядкового условия:
    - Мержим с таблицей последовательностей
    - Мержим с инфой о вершинах
    - Выбираем строки
- Строим последовательность на базе сохраненных строк

Как это выглядит
user_id 